In [5]:
import yfinance as yf
import pandas as pd
from datetime import datetime
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

# 데이터 수집
start_date = '2021-01-01'
end_date = datetime.now().strftime('%Y-%m-%d')
df = yf.download('BTC-USD', start=start_date, end=end_date)

# 상승(1) 또는 하락(0) 레이블 추가
df['Target'] = (df['Close'].shift(-1) > df['Close']).astype(int)
df.dropna(inplace=True)

# 피처와 레이블 설정
X = df[['Close', 'Volume']]
y = df['Target']

# 데이터 분할
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 모델 학습
model = LogisticRegression()
model.fit(X_train, y_train)

# 예측
y_pred = model.predict(X_test)

# 평가
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy:.2f}')


[*********************100%%**********************]  1 of 1 completed

Accuracy: 0.41


In [6]:
df

,Open,High,Low,Close,Adj Close,Volume,Target
Date,,,,,,,
2021-01-01,28994.009766,29600.626953,28803.585938,29374.152344,29374.152344,40730301359,1
2021-01-02,29376.455078,33155.117188,29091.181641,32127.267578,32127.267578,67865420765,1
2021-01-03,32129.408203,34608.558594,32052.316406,32782.023438,32782.023438,78665235202,0
2021-01-04,32810.949219,33440.218750,28722.755859,31971.914062,31971.914062,81163475344,1
2021-01-05,31977.041016,34437.589844,30221.187500,33992.429688,33992.429688,67547324782,1
...,...,...,...,...,...,...,...
2024-06-15,66006.742188,66402.187500,65871.773438,66191.000000,66191.000000,14121265576,1
2024-06-16,66189.359375,66894.843750,66018.250000,66639.046875,66639.046875,13281140541,0
2024-06-17,66636.515625,67188.320312,65094.964844,66490.296875,66490.296875,30006354476,0
